# Parsing (Extract)

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from datetime import date
from openpyxl import load_workbook


url = 'https://www.worldometers.info/coronavirus/'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

soup = BeautifulSoup(webpage, "html.parser")


table = soup.find('table')
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    res.append(row)   

In [2]:
len(res)

236

In [3]:
# An example of how 'res' holds data
res[:3]

[[],
 ['',
  '\nNorth America\n',
  '13,441,980',
  '+44,784',
  '377,471',
  '+456',
  '8,484,504',
  '+25,118',
  '4,580,005',
  '24,671',
  '',
  '',
  '',
  '',
  '',
  'North America',
  '\n',
  '',
  ''],
 ['',
  '\nAsia\n',
  '15,154,082',
  '+73,022',
  '267,559',
  '+1,305',
  '13,530,134',
  '+66,099',
  '1,356,389',
  '24,777',
  '',
  '',
  '',
  '',
  '',
  'Asia',
  '\n',
  '',
  '']]

# Transpose

In [4]:
x = """idx
Country
Total Cases
New Cases
Total Deaths
New Deaths
Total Recovered
New Recovered
Active Cases
Critical
Cases/1M pop
Deaths/1M pop
Total Tests
Tests/1M pop
Population
Continent
1 Case every X ppl
1 Death every X ppl
1 Test every X ppl""".split('\n')


data = pd.DataFrame(res,columns = x)

In [5]:
print(data)

      idx            Country Total Cases New Cases Total Deaths New Deaths  \
0    None               None        None      None         None       None   
1          \nNorth America\n  13,441,980   +44,784      377,471       +456   
2                   \nAsia\n  15,154,082   +73,022      267,559     +1,305   
3          \nSouth America\n  10,359,430    +1,428      310,055        +52   
4                 \nEurope\n  14,082,135  +142,456      324,405     +2,020   
..    ...                ...         ...       ...          ...        ...   
231                   Total:  14,082,135  +142,456      324,405     +2,020   
232                   Total:   1,990,738    +3,006       47,491        +53   
233                   Total:      42,143       +22          994              
234                   Total:         721                     15              
235                   Total:  55,071,229  +264,718    1,327,990     +3,886   

    Total Recovered New Recovered Active Cases Critical Cases/1

In [6]:
data.drop(0,inplace=True)

data.set_index('Country',inplace=True)

# Removing 'world' row
data.drop(['World','Total:'],inplace=True)

continents = [i for i in data.index if '\n' in i]
data.drop(continents,inplace=True)

In [7]:
data.drop("""idx
Population
Continent
1 Case every X ppl
1 Death every X ppl
1 Test every X ppl
New Recovered""".split('\n'),axis=1,inplace=True)

In [8]:
#pathCoronaOld = r'C:\Users\97250\Desktop\coronaWeb\coronaWorld.xlsx'
#oldCountries = pd.read_excel(pathCoronaOld, sheet_name=0)['Country'].values
#newCountries = data.index

In [9]:
def commaRemover(x):
    
    if type(x) == str:
        
        if x == ' ' or x == '' or x == 'N/A' : # in list is better
            x = 0
                    
        else:
            x = x.replace(',','').replace("+","")
            x = float(x)
    
    return x
    
for i in data.columns:
    data[i] = data[i].apply(commaRemover)

#adding new columns
#have to watch infinties of no cases are reported

In [10]:
#perhaps this:
data['Deaths/Cases'] =  data[['Total Deaths','Total Cases']].apply(lambda x : round(int(x[0])/int(x[1]),2) if x[1] !='' else 0 ,axis=1)
data['Tests/Cases'] =  data[['Total Tests','Total Cases']].apply  (lambda x : round(x[0]/x[1],2) if x[1] !=0 else 1000 ,axis=1)

In [11]:
# Fill NA's
data.fillna(value = 0,inplace =True)

In [12]:
data

,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Critical,Cases/1M pop,Deaths/1M pop,Total Tests,Tests/1M pop,Deaths/Cases,Tests/Cases
Country,,,,,,,,,,,,,
USA,11404516.0,37302.0,252026.0,125.0,6954144.0,4198346.0,20709.0,34379.0,760.0,168438066.0,507752.0,0.02,14.77
India,8868468.0,22851.0,130417.0,308.0,8282065.0,455986.0,8944.0,6403.0,94.0,125698525.0,90751.0,0.01,14.17
Brazil,5863093.0,0.0,165811.0,0.0,5291511.0,405771.0,8318.0,27510.0,778.0,21900000.0,102756.0,0.03,3.74
France,1981827.0,0.0,44548.0,0.0,139810.0,1797469.0,4896.0,30337.0,682.0,19005625.0,290926.0,0.02,9.59
Russia,1948603.0,22778.0,33489.0,303.0,1453849.0,461265.0,2300.0,13350.0,229.0,69111898.0,473505.0,0.02,35.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Anguilla,3.0,0.0,0.0,0.0,3.0,0.0,0.0,199.0,0.0,1820.0,120906.0,0.00,606.67
Marshall Islands,1.0,0.0,0.0,0.0,1.0,0.0,0.0,17.0,0.0,0.0,0.0,0.00,0.00
Vanuatu,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.00,0.00


In [13]:

today = str(date.today())

path = r'C:\Users\97250\Desktop\coronaWeb\coronaWorld.xlsx'
book = load_workbook(path)

### checks if no sheet from today
last_sheet = book.get_sheet_names()[-1]
if last_sheet==today:
    book.remove_sheet(book.get_sheet_by_name(last_sheet))

writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book    
data.to_excel(writer ,sheet_name = today) #the sheet name will be the str(current date)
writer.save()
writer.close()

<ipython-input-13-5962503f643f>:7: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  last_sheet = book.get_sheet_names()[-1]


In [14]:
#checks sheets
file = pd.read_excel(path, sheet_name=None)

sheets = [i for i in file][-10:]

sheets

['2020-05-10',
 '2020-05-12',
 '2020-05-14',
 '2020-05-15',
 '2020-05-18',
 '2020-10-19',
 '2020-10-25',
 '2020-11-05',
 '2020-11-15',
 '2020-11-16']